In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import Image
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud
import cv2 as cv

import src.models.Pix2Vox.utils.binvox_rw as br
from settings import *


In [13]:
taxonomy_id = "02691156"
test_id = "1a04e3eab45ca15dd86060f189eb133"

result_dir = os.path.join(SHAPENET_DATASET_DIR, "results", "sfm", taxonomy_id, test_id)
binvox_result_path = os.path.join(result_dir, "omvs", "scene_dense.binvox")

binvox_truth_path = os.path.join(SHAPENET_DATASET_DIR, "ShapeNetVox32", taxonomy_id, test_id, "model.binvox") 
truth_vox = br.read_as_3d_array(open(binvox_result_path, "rb"))
truth_vox.translate


[0.0, 0.0, 0.0]

In [11]:
file_name = "scene_dense"
cloud = PyntCloud.from_file(os.path.join(SHAPENET_DATASET_DIR, f"results/sfm/02691156/1a04e3eab45ca15dd86060f189eb133/omvs/{file_name}.ply"))

# cloud.plot(mesh=True, backend="threejs")

voxelgrid_id = cloud.add_structure("voxelgrid", n_x=32, n_y=32, n_z=32)
voxelgrid = cloud.structures[voxelgrid_id]
# voxelgrid.plot(d=3, mode="density", cmap="hsv")

x_cords = voxelgrid.voxel_x
y_cords = voxelgrid.voxel_y
z_cords = voxelgrid.voxel_z

voxel = np.zeros((32, 32, 32)).astype(np.bool)

for x, y, z in zip(x_cords, y_cords, z_cords):
    voxel[x][y][z] = True

with open(os.path.join(SHAPENET_DATASET_DIR, f"results/sfm/02691156/1a04e3eab45ca15dd86060f189eb133/{file_name}.binvox"), 'wb') as f:
    v = br.Voxels(voxel, (32, 32, 32), (0, 0, 0), 1, 'xyz')
    v.write(f)

In [12]:
# rendering_metadata header: azimuth, elevation, in-plane rotation, distance, the field of view.

In [43]:
import math
import scipy as sp
metadata = np.array([
    [322.076852257, 25.631301418, 0.652915183775],
    [40.3117661825, 25.8383382351, 0.942380338756],
    [95.051584931, 28.273303161, 0.830560386203],
    [194.156286829, 27.5944161956, 0.934022406951],
    [119.197787507, 25.9090378072, 0.793442208555],
    [209.173082983, 25.2847501512, 0.878735772563],
    [313.487592473, 26.9180334858, 0.839838467479],
    [181.843401728, 26.1342125477, 0.713165983397],
    [98.7037137443, 25.8779360358, 0.786218303718],
    [103.500649825, 29.7254537011, 0.704913513573],
    [45.8458163791, 29.8654674711, 0.929834770812],
    [202.04133569, 29.2663432613, 0.843428175715],
    [317.313478058, 28.9747899229, 0.949127982751],
    [184.362766483, 27.4752727159, 0.751438473761],
    [118.193308902, 25.8499061741, 0.92684023909],
    [281.405721426, 29.40360047, 0.720646023312],
    [242.090618091, 28.6942715164, 0.859843946922],
    [222.52404222, 25.5463411809, 0.821500110181],
    [214.431816242, 26.3060659391, 0.653413194421],
    [174.643709228, 28.2573661685, 0.826953547324],
    [93.0729431317, 26.302795738, 0.83229297474],
    [70.0454078923, 25.1137568739, 0.695374310276],
    [162.718217671, 26.487929586, 0.907247135638],
    [352.726929932, 25.8652666583, 0.893896836271]])


def angles_to_matrix(azi, ele, rol):
    # element1 = (np.cos(rol) * np.cos(azi) - np.sin(rol) * np.cos(ele) * np.sin(azi))
    # element2 = (np.sin(rol) * np.cos(azi) + np.cos(rol) * np.cos(ele) * np.sin(azi))
    # element3 = (np.sin(ele) * np.sin(azi))
    # element4 = (-np.cos(rol) * np.sin(azi) - np.sin(rol) * np.cos(ele) * np.cos(azi))
    # element5 = (-np.sin(rol) * np.sin(azi) + np.cos(rol) * np.cos(ele) * np.cos(azi))
    # element6 = (np.sin(ele) * np.cos(azi))
    # element7 = (np.sin(rol) * np.sin(ele))
    # element8 = (-np.cos(rol) * np.sin(ele))
    # element9 = (np.cos(ele))
    
    # r_azi = np.array([
    #     [np.cos(azi), 0, np.sin(azi)],
    #     [0, 1, 0],
    #     [-np.sin(azi), 0, np.cos(azi)]
    # ]) 
    # r_ele = np.array([
    #     [np.cos(ele), -np.sin(ele), 0],
    #     [np.sin(ele), np.cos(ele), 0],
    #     [0, 0, 1]
    # ])  
    # r_rol = np.array([
    #     [1, 0, 0],
    #     [0, np.cos(rol), -np.sin(rol)],
    #     [0, np.sin(rol), np.cos(rol)]
    # ]) 

    r_azi = np.array([
        [np.cos(azi), 0, np.sin(azi)],
        [0, 1, 0],
        [-np.sin(azi), 0, np.cos(azi)]
    ]) # pitch
    r_ele = np.array([
        [np.cos(ele), -np.sin(ele), 0],
        [np.sin(ele), np.cos(ele), 0],
        [0, 0, 1]
    ])  # yaw
    r_rol = np.array([
        [1, 0, 0],
        [0, np.cos(rol), -np.sin(rol)],
        [0, np.sin(rol), np.cos(rol)]
    ]) # roll
        
        
    R = ((r_azi @ r_ele) @ r_rol)

    return R.tolist()
    # return [
    #     [element1, element2, element3],
    #     [element4, element5, element6],
    #     [element7, element8, element9]]

def obj_centened_camera_pos(azimuth_deg, elevation_deg, dist):
    phi = float(elevation_deg) / 180 * math.pi
    theta = float(azimuth_deg) / 180 * math.pi
    x = (dist * math.cos(theta) * math.cos(phi))
    y = (dist * math.sin(theta) * math.cos(phi))
    z = (dist * math.sin(phi))
    return [x, y, z]

extrinsics = []
for idx, m in enumerate(metadata):
    center = obj_centened_camera_pos(m[0], m[1], m[2])
    rotation = angles_to_matrix(m[0], m[1], 0)
    extrinsic = {
        "key": idx,
        "value": {
            "rotation": rotation,
            "center": center
        }
    }

    extrinsics.append(extrinsic)
print(extrinsics)
# r = sp.spatial.transform.Rotation.from_euler('xyz', [
    # []
# ])
# "key": 19,
# "value": {
#     "rotation": [
#         [
#             0.955816082322138,
#             -0.11956428567202157,
#             0.26855166796372539
#         ],
#         [
#             0.10015865761269897,
#             0.9913439064754334,
#             0.08488523074863716
#         ],
#         [
#             -0.2763763015882185,
#             -0.054236894138275048,
#             0.9595178472726037
#         ]
#     ],
#     "center": [
#         0.7479031729311192,
#         -0.16503236187189367,
#         -0.6542228640308786
#     ]
# }

[{'key': 0, 'value': {'rotation': [[-0.05582505924310505, 0.030393084914404463, 0.9979778670641397], [0.47816148932428215, 0.8782719340427453, 0.0], [-0.8764959514382757, 0.4771945832280595, -0.06356238549732372]], 'center': [0.4643607940384801, -0.36179640719196016, 0.28243698395727806]}}, {'key': 1, 'value': {'rotation': [[-0.6571981040348028, 0.5598681172374114, 0.5046170264211233], [0.648488400980331, 0.7612245357277795, 0.0], [-0.38412686165775217, 0.3272382885712837, -0.8633433017322271]], 'center': [0.6467582574558325, 0.5487192703171204, 0.4107208561410236]}}, {'key': 2, 'value': {'rotation': [[-0.6939715311309603, -0.0007152914976831503, 0.7200020849538034], [0.0010307211256339209, -0.9999994688068395, 0.0], [0.7200017024936203, 0.0007421213594623541, 0.693971899764087]], 'center': [-0.06440810567679436, 0.728631833802544, 0.39341810052709214]}}, {'key': 3, 'value': {'rotation': [[-0.6317794438729624, -0.5108096287056872, -0.5830336675708603], [0.6287290173182118, -0.777624474

In [ ]:
# ./openMVG_main_ConvertSfM_DataFormat -i /mnt/d/Projekty/Programowanie/Studia/Semestr9/cv/AOiW_project_3d_modelling/data/ShapeNet/results/sfm/02691156/1a04e3eab45ca15dd86060f189eb133/reconstruction_global/sfm_data.bin -I -V -E -o ./camera_poses.json
# /opt/openmvg/bin/openMVG_main_ComputeStructureFromKnownPoses -i ./camera_poses.json -d -m ./matches/ -f ./matches/matches.e.bin -o ./reconstruction_global/sfm_data_structure.bin
# /opt/openmvg/bin/openMVG_main_openMVG2openMVS -i ./reconstruction_global/sfm_data_structure.bin -o ./omvs_structure/scene.mvs -d ./omvs_structure
# /opt/openmvs/bin/OpenMVS/DensifyPointCloud ./omvs_structure/scene.mvs -v 0